In [232]:
#importing libraries
from os import getcwd
from pandas import *
import geopandas as gpd
from plotnine import *
import re
from statistics import count

In [233]:
path_to_data = getcwd() + r"\data_science_salaries.csv"

data = read_csv(path_to_data, sep = ",")

print("Number of rows: ", len(data))

Number of rows:  6599


In [234]:
x = data["job_title"].value_counts()
print(x)
print("\n Sum of Data Engineers, Data Scientists, Data Analyst and ML Engineers: ", sum(x[0:4]))

job_title
Data Engineer                      1307
Data Scientist                     1243
Data Analyst                        910
Machine Learning Engineer           629
Analytics Engineer                  246
                                   ... 
Data Science Tech Lead                1
Staff Machine Learning Engineer       1
Cloud Data Architect                  1
Marketing Data Engineer               1
Sales Data Analyst                    1
Name: count, Length: 132, dtype: int64

 Sum of Data Engineers, Data Scientists, Data Analyst and ML Engineers:  4089


In [235]:
print(data.head())

        job_title experience_level employment_type work_models  work_year  \
0   Data Engineer        Mid-level       Full-time      Remote       2024   
1   Data Engineer        Mid-level       Full-time      Remote       2024   
2  Data Scientist     Senior-level       Full-time      Remote       2024   
3  Data Scientist     Senior-level       Full-time      Remote       2024   
4    BI Developer        Mid-level       Full-time     On-site       2024   

  employee_residence  salary salary_currency  salary_in_usd company_location  \
0      United States  148100             USD         148100    United States   
1      United States   98700             USD          98700    United States   
2      United States  140032             USD         140032    United States   
3      United States  100022             USD         100022    United States   
4      United States  120000             USD         120000    United States   

  company_size  
0       Medium  
1       Medium  
2    

In [236]:
print(type(data["job_title"].dtype))
print(type(data["work_models"].dtype))

<class 'numpy.dtypes.ObjectDType'>
<class 'numpy.dtypes.ObjectDType'>


In [237]:
for x in data:
    if data[x].dtype == "object":
        data[x] = data[x].str.lower()

jobs_to_keep = ["data engineer", "data scientist", "data analyst", "machine learning engineer"]
data = data[data["job_title"].isin(jobs_to_keep)]


print(unique(data["job_title"]))

['data engineer' 'data scientist' 'data analyst'
 'machine learning engineer']


In [ ]:
print("Number of rows: ",len(data))

Number of rows:  4089
        job_title experience_level employment_type work_models  work_year  \
0   data engineer        mid-level       full-time      remote       2024   
1   data engineer        mid-level       full-time      remote       2024   
2  data scientist     senior-level       full-time      remote       2024   
3  data scientist     senior-level       full-time      remote       2024   
8   data engineer  executive-level       full-time      remote       2024   

  employee_residence  salary salary_currency  salary_in_usd company_location  \
0      united states  148100             usd         148100    united states   
1      united states   98700             usd          98700    united states   
2      united states  140032             usd         140032    united states   
3      united states  100022             usd         100022    united states   
8      united states  219650             usd         219650    united states   

  company_size  
0       medium  


In [239]:

data["experience_level"] = data["experience_level"].str.replace("-level","")
print(data["experience_level"])


0             mid
1             mid
2          senior
3          senior
8       executive
          ...    
6589          mid
6591          mid
6592        entry
6597          mid
6598       senior
Name: experience_level, Length: 4089, dtype: object


In [247]:
for x in data:
    print(x,":",data[x].hasnans)


job_title : False
experience_level : False
employment_type : False
work_models : False
work_year : False
employee_residence : False
salary : False
salary_currency : False
salary_in_usd : False
company_location : False
company_size : False
